In [1]:
from copy import deepcopy

import numpy as np
import pandas as pd

import pyvinecopulib as pv
from vcimpute.helper_datagen import make_complete_data_matrix, mask_MCAR
from vcimpute.helper_subvines import find_subvine_structures, remove_column, remove_inbetween
from vcimpute.utils import make_triangular_array, is_leaf_in_all_subtrees

In [2]:
class VineCopFit:
    pass

In [3]:
X = make_complete_data_matrix(n=1000,d=10,copula_type='gaussian')
X_mis = mask_MCAR(X, d_mis=4, mask_fraction=.2, monotonic_missingness=True)
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian], num_threads=10)
cop = pv.Vinecop(data=X_mis, controls=controls)

In [4]:
T = cop.matrix
pcs = cop.pair_copulas

In [5]:
miss_vars, = np.where(np.count_nonzero(np.isnan(X_mis), axis=0))
miss_vars += 1
miss_vars = miss_vars.astype(np.uint64)

In [6]:
last_impute_var = None
for var_mis in miss_vars:
    if is_leaf_in_all_subtrees(T, var_mis):
        last_impute_var = var_mis
if last_impute_var:
    miss_vars = miss_vars[miss_vars != last_impute_var]

In [7]:
find_subvine_structures(T, pcs, 7)

[(array([[3, 1, 4, 4, 4],
         [1, 4, 7, 7, 0],
         [4, 7, 1, 0, 0],
         [7, 3, 0, 0, 0],
         [9, 0, 0, 0, 0]], dtype=uint64),
  [[<pyvinecopulib.Bicop>
    Gaussian, parameters = -0.685432,
    <pyvinecopulib.Bicop>
    Gaussian, parameters = -0.572715,
    <pyvinecopulib.Bicop>
    Gaussian, parameters = -0.343964,
    <pyvinecopulib.Bicop>
    Gaussian, parameters = -0.424838],
   [<pyvinecopulib.Bicop>
    Gaussian, parameters = 0.237991,
    <pyvinecopulib.Bicop>
    Gaussian, parameters = -0.257414,
    <pyvinecopulib.Bicop>
    Gaussian, parameters = 0.081314],
   [<pyvinecopulib.Bicop>
    Gaussian, parameters = 0.280469,
    <pyvinecopulib.Bicop>
    Gaussian, parameters = -0.366956],
   [<pyvinecopulib.Bicop>
    Gaussian, parameters = 0.338044]]),
 (array([[1, 4, 4, 4],
         [4, 7, 7, 0],
         [7, 1, 0, 0],
         [3, 0, 0, 0]], dtype=uint64),
  [[<pyvinecopulib.Bicop>
    Gaussian, parameters = -0.572715,
    <pyvinecopulib.Bicop>
    Gaussian, 